ODR Sequential Serial Bias
==========================



## Imports



## Notebook Settings



    The autoreload extension is already loaded. To reload it, use:
      %reload_ext autoreload
    Python exe
    /home/leon/mambaforge/envs/torch/bin/python



## Helpers



## Model



    seed 1

    1050
    450
    EE post 0 pre 0 Jab tensor(0.0759, device='cuda:1') torch.Size([1050, 1050])

    ---------------------------------------------------------------------------
    RuntimeError                              Traceback (most recent call last)
    Cell In[12], line 2
          1 model_state_dict = torch.load('../models/odr/odr_%d.pth' % seed)
    ----> 2 model.load_state_dict(model_state_dict);
          3 model.eval();
    
    File ~/mambaforge/envs/torch/lib/python3.10/site-packages/torch/nn/modules/module.py:2153, in Module.load_state_dict(self, state_dict, strict, assign)
       2148         error_msgs.insert(
       2149             0, 'Missing key(s) in state_dict: {}. '.format(
       2150                 ', '.join(f'"{k}"' for k in missing_keys)))
       2152 if len(error_msgs) > 0:
    -> 2153     raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
       2154                        self.__class__.__name__, "\n\t".join(error_msgs)))
       2155 return _IncompatibleKeys(missing_keys, unexpected_keys)
    
    RuntimeError: Error(s) in loading state_dict for Network:
    	size mismatch for Wab_train: copying a param with shape torch.Size([750, 750]) from checkpoint, the shape in current model is torch.Size([1050, 1050]).

    Parameter containing:
    tensor(1.9578, device='cuda:1', requires_grad=True)

![img](./figures/odr_seq_sb/figure_14.png)

    controls_delay_rand



## Simulating Consecutive Trials



    torch.Size([768, 805, 1500]) torch.Size([768, 151, 1050])

    torch.Size([768, 805, 1500]) torch.Size([768, 151, 1050])

    100% 100/100 [03:39<00:00,  2.20s/it]

    rates (100, 768, 151, 1050)
    curr (100, 768, 1) prev (100, 768, 1) iti (76800,)
    start torch.Size([100, 2, 768]) end torch.Size([100, 2, 768]) delay (76800,)

    (2, 768) (2, 768)

    (100, 768, 151)

    1.4875381112098642

![img](./figures/odr_seq_sb/figure_23.png)

![img](./figures/odr_seq_sb/figure_24.png)

    saving to ./models/nih//rates_list_controls_delay_rand.pkl
    saving to ./models/nih//curr_list_controls_delay_rand.pkl
    saving to ./models/nih//prev_list_controls_delay_rand.pkl
    saving to ./models/nih//delay_list_controls_delay_rand.pkl
    saving to ./models/nih//start_idx_controls_delay_rand.pkl
    saving to ./models/nih//end_idx_controls_delay_rand.pkl
    saving to ./models/nih//phi_list_controls_delay_rand.pkl



## Errors



    (50, 768, 1) (50, 768, 1)

    (50, 768, 151) (50, 768, 1)

    (38400, 1) (38400, 1) (50, 768, 151)

    torch.Size([100, 2, 768])
    (38400, 1)

    torch.Size([100, 2, 768])
    (38400, 1)

![img](./figures/odr_seq_sb/figure_36.png)

![img](./figures/odr_seq_sb/figure_37.png)



## Biases



    (38400, 1) (38400, 1) (38400, 1)

![img](./figures/odr_seq_sb/figure_43.png)



## Delay Dependency



    torch.Size([100, 2, 768])
    (76800, 1)

       target_loc    rel_loc      errors  delay  prev_delay
    0  237.999985  76.000023 -119.624756    6.0         0.0
    1  193.000000  29.000000   -6.494704    0.0         0.0
    2  168.000000  12.000000  174.521240    2.0         4.0
    3  306.000000  63.000042   99.572754    4.0         0.0
    4  129.000000 -81.999992   84.549286    6.0         8.0

    76800 [0. 2. 4. 6. 8.]

![img](./figures/odr_seq_sb/figure_48.png)

    [0. 2. 4. 6. 8.] 76800

![img](./figures/odr_seq_sb/figure_49.png)

![img](./figures/odr_seq_sb/figure_51.png)

    [0. 2. 4. 6. 8.] 76800

![img](./figures/odr_seq_sb/figure_52.png)

